In [10]:
from operator import itemgetter
from os import path

from lib.babi import read_task


def print_dataset_stats(in_dataset):
    print '#Dialogs:\t{}'.format(
        len(in_dataset)
    )
    dialogues_only = map(itemgetter(1), in_dataset)
    print 'Mean #turns:\t{0:.3f}'.format(
        sum(map(len, dialogues_only)) / float(len(in_dataset))
    )
    system_turns = reduce(
        lambda x, y: x + filter(lambda turn: turn['agent'] == 'system', y),
        dialogues_only,
        []
    )
    user_turns = reduce(
        lambda x, y: x + filter(lambda turn: turn['agent'] == 'user', y),
        dialogues_only,
        []
    )
    print 'Mean system turn length (words):    {0:.3f}'.format(
        sum(map(lambda turn: len(turn['text'].split()), system_turns)) / float(len(system_turns))
    )
    print 'Mean user turn length (words):    {0:.3f}'.format(
        sum(map(lambda turn: len(turn['text'].split()), user_turns)) / float(len(user_turns))
    )

In [11]:
DATA_FOLDER = 'dialog-bAbI-tasks'
task1_train = read_task(path.join(DATA_FOLDER, 'dialog-babi-task1-API-calls-trn.txt'))
task1_dev = read_task(path.join(DATA_FOLDER, 'dialog-babi-task1-API-calls-dev.txt'))
task1_test = read_task(path.join(DATA_FOLDER, 'dialog-babi-task1-API-calls-tst.txt'))
task1_oov_test = read_task(path.join(DATA_FOLDER, 'dialog-babi-task1-API-calls-tst-OOV.txt'))

General stats
==

In [12]:
print 'Task 1 stats:'
print 'train'
print_dataset_stats(task1_train)
print 'dev'
print_dataset_stats(task1_dev)
print 'test'
print_dataset_stats(task1_test)
print 'test_oov'
print_dataset_stats(task1_oov_test)

Task 1 stats:
train
#Dialogs:	1000
Mean #turns:	12.048
Mean system turn length (words):    6.256
Mean user turn length (words):    3.798
dev
#Dialogs:	1000
Mean #turns:	12.030
Mean system turn length (words):    6.245
Mean user turn length (words):    3.796
test
#Dialogs:	1000
Mean #turns:	11.872
Mean system turn length (words):    6.242
Mean user turn length (words):    3.835
test_oov
#Dialogs:	1000
Mean #turns:	12.040
Mean system turn length (words):    6.258
Mean user turn length (words):    3.787


API call stats
==

In [13]:
def get_answers(in_dataset):
    return reduce(
        lambda x, y: x + y,
        [
            map(itemgetter('text'), filter(lambda turn: turn['agent'] == 'system', dialogue))
            for dialogue_name, dialogue in in_dataset
        ],
        []
    )

def get_candidates_list(data_dir):
    candidates_file = path.join(data_dir, 'dialog-babi-candidates.txt')
    with open(candidates_file) as candidates_in:
        return [line.strip().split(' ', 1)[1] for line in candidates_in]

In [14]:
train_answers = get_answers(task1_train)
print 'train'
print '#unique answers (intelocutory): {}'.format(
    len(
        set(
            filter(lambda x: not x.startswith('api_call'), train_answers)
        )
    )
)
unique_api_calls_train = set(filter(lambda x: x.startswith('api_call'), train_answers))
print '#unique answers (api call): {}'.format(len(unique_api_calls_train))

test_answers = get_answers(task1_test)
print 'test'
print '#unique answers (intelocutory): {}'.format(
    len(
        set(
            filter(lambda x: not x.startswith('api_call'), test_answers)
        )
    )
)
unique_api_calls_test = set(filter(lambda x: x.startswith('api_call'), test_answers))
print '#unique answers (api call): {}'.format(len(unique_api_calls_test))

print '#test api calls not present in the trainset: {}'.format(len(unique_api_calls_test.difference(unique_api_calls_train)))

train
#unique answers (intelocutory): 7
#unique answers (api call): 159
test
#unique answers (intelocutory): 7
#unique answers (api call): 141
#test api calls not present in the trainset: 141


In [16]:
answer_candidates = get_candidates_list(DATA_FOLDER)

In [19]:
all_answers = reduce(
    lambda x, y: x + y,
    map(get_answers, [task1_train, task1_dev, task1_test, task1_oov_test])
)

In [20]:
unique_api_calls_all = set(filter(lambda x: x.startswith('api_call'), all_answers))
print '#unique answers (api call): {}'.format(len(unique_api_calls_all))

print '#all api calls not present in the candidates_list: {}'.format(len(unique_api_calls_all.difference(answer_candidates)))

#unique answers (api call): 460
#all api calls not present in the candidates_list: 0


In [27]:
unique_api_calls_all.difference(answer_candidates)

set()